In [36]:
import hues
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, CrossEntropyLoss

In [37]:
DATA_DIR = '../05-Transforms/data/'

In [38]:
transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_data = torchvision.datasets.CIFAR10(
    root=DATA_DIR,
    transform=transformer,
    train=True,
    download=True
)

test_data = torchvision.datasets.CIFAR10(
    root=DATA_DIR,
    transform=transformer,
    train=False,
    download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
len(train_data), len(test_data)

(50000, 10000)

In [5]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_data,
    batch_size=64,
    shuffle=True,
    num_workers=2,
    drop_last=True
)

In [41]:
len(train_loader.dataset), len(train_loader)

(50000, 781)

In [43]:
test_loader[0]

TypeError: 'DataLoader' object is not subscriptable

In [6]:
vgg_pretrained = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1)
vgg = torchvision.models.vgg16()

In [7]:
# vgg输出为1000，而CIFAR10数据集输出为10，因此最后还需要添加一个线性层
# vgg_pretrained.add_module('Linear', torch.nn.Linear(1000, 10))
vgg_pretrained.classifier.add_module('Linear', torch.nn.Linear(1000, 10))
vgg_pretrained

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
# 除了上述方法，在模型的最后新添加一个线性层外，也可以对模型的最后一个线性层进行修改，使其输出为10也行。
vgg.classifier[6] = torch.nn.Linear(4096, 10)
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
# 创建网络模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg_pretrained.to(device)
print(vgg_pretrained)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
# 损失函数
loss_fun = CrossEntropyLoss()
# 转移到GPU上
loss_fun.to(device)

# 优化器
learning_rate = 0.0001
optimizer = torch.optim.SGD(vgg_pretrained.parameters(), lr=learning_rate)

# 使用tensorboard可视化网络结构
writer = SummaryWriter('./logs')

In [11]:
%%time
for epoch in range(50):
    epoch_train_loss = 0  # 对每一轮的loss进行累加，观察经过多轮学习后，loss是否有下降
    epoch_test_loss = 0
    train_right_num = 0
    pred_right_num = 0  # 测试集中预测正确的数量

    # 对训练集的每一个批量数据进行训练
    # module.train()  # 这一行代码可有可无，只针对特定模型需要设置，官方文档：https://blog.csdn.net/u014764291/article/details/105924182
    for i, data in enumerate(train_loader):
        imgs, targets = data
        # 将数据转移到GPU
        imgs = imgs.to(device)
        targets = targets.to(device)

        outputs = vgg_pretrained(imgs)
        res_loss = loss_fun(outputs, targets)
        optimizer.zero_grad()
        res_loss.backward()
        optimizer.step()
        epoch_train_loss += res_loss

        train_right_num += (outputs.argmax(1) == targets).sum()

    # 检验当前模型在测试集上的效果
    # module.eval()  # 参考module.train()
    with torch.no_grad():  # 在测试集上检验效果，不需要进行梯度下降优化
        for i, data in enumerate(test_loader):
            imgs, targets = data
            # 将数据转移到GPU
            imgs = imgs.to(device)
            targets = targets.to(device)

            outputs = vgg_pretrained(imgs)
            res_loss = loss_fun(outputs, targets)
            epoch_test_loss += res_loss

            item_train_num = epoch * len(data) + i + 1

            pred_right_num += (outputs.argmax(1) == targets).sum()

    torch.save(vgg_pretrained, f'./module/module_{epoch}.pth')
    writer.add_scalar('Loss/Train', epoch_train_loss.item(), epoch)
    writer.add_scalar('Loss/Test', epoch_test_loss.item(), epoch)
    writer.add_scalar('Accuracy/Train', train_right_num / len(train_data), epoch)
    writer.add_scalar('Accuracy/Test', pred_right_num / len(test_data), epoch)

    hues.info(
        f'第{epoch}轮的训练集累积Loss值为：{epoch_train_loss.item()}，测试集累积Loss值为：{epoch_test_loss.item()}，测试集上的预测正确率为{pred_right_num / len(test_data)}。')

# tensorboard --logdir='logs'

15:12:06 - INFO - 第0轮的训练集累积Loss值为：1391.1151123046875，测试集累积Loss值为：225.072265625，测试集上的预测正确率为0.4885999858379364。
15:12:44 - INFO - 第1轮的训练集累积Loss值为：1032.6259765625，测试集累积Loss值为：191.50112915039062，测试集上的预测正确率为0.5649999976158142。
15:13:23 - INFO - 第2轮的训练集累积Loss值为：907.9688720703125，测试集累积Loss值为：173.18649291992188，测试集上的预测正确率为0.6053999662399292。
15:13:58 - INFO - 第3轮的训练集累积Loss值为：826.86669921875，测试集累积Loss值为：162.69613647460938，测试集上的预测正确率为0.6359999775886536。
15:14:33 - INFO - 第4轮的训练集累积Loss值为：773.4359130859375，测试集累积Loss值为：153.57334899902344，测试集上的预测正确率为0.6536999940872192。
15:15:09 - INFO - 第5轮的训练集累积Loss值为：730.9052124023438，测试集累积Loss值为：145.92526245117188，测试集上的预测正确率为0.6708999872207642。
15:15:46 - INFO - 第6轮的训练集累积Loss值为：692.529296875，测试集累积Loss值为：141.09327697753906，测试集上的预测正确率为0.683899998664856。
15:16:25 - INFO - 第7轮的训练集累积Loss值为：665.0726928710938，测试集累积Loss值为：136.00230407714844，测试集上的预测正确率为0.6998999714851379。
15:17:03 - INFO - 第8轮的训练集累积Loss值为：642.4857177734375，测试集累积Loss值为：132.40689086914062，测试集上的预测正确率为0.70480

In [18]:
imgs = None
for i, data in enumerate(test_loader):
    imgs, targets = data
    imgs = imgs.to(device)
    break

writer.add_graph(vgg_pretrained, imgs)
writer.close()

In [19]:
# 模型的保存：保存模型结构+模型参数
torch.save(vgg_pretrained, 'SavedModule.pth')

In [20]:
# 模型的加载
model = torch.load('./SavedModule.pth')
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
# 第二种保存方式：只保存模型参数
torch.save(vgg_pretrained.state_dict(), 'SavedStateDict.pth')

In [23]:
# 模型的加载
model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1)
model.classifier.add_module('Linear', torch.nn.Linear(1000, 10))
model.load_state_dict(torch.load('SavedStateDict.pth'))
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1